In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import shutil
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.globals import ThemeType
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
basic_info = load_data("产品目录.xlsx")
basic_info

,strategy,fund_code,fund_name,benchmark_code,benchmark_name,threshold
0,主观CTA,SM6151,草本优益1号,NH0100.NHF,南华商品指数,-0.10
1,主观CTA,SM6151,草本致远1号,NH0300.NHF,南华农产品指数,-0.10
2,主观CTA,SGC034,济海实恒对冲二号,NH0100.NHF,南华商品指数,-0.10
3,主观CTA,ST6571,星恒哲萌1号,NH0100.NHF,南华商品指数,-0.10
4,量化CTA,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.10
5,量化CTA,SXF557,千衍九凌1号,NH0100.NHF,南华商品指数,-0.10
6,量化CTA,SCV067,众壹资产量合兴成一号,NH0100.NHF,南华商品指数,-0.10
7,量化CTA,SLS182,弘源多元化CTA,NH0100.NHF,南华商品指数,-0.10
8,量化CTA,SN3221,宏锡量化CTA7号,NH0100.NHF,南华商品指数,-0.10
9,量化CTA,SGA623,会世元丰CTA1号,NH0100.NHF,南华商品指数,-0.10


In [3]:
# 删除html文件
def delete_html_files(directory):
    # 遍历目录及其子目录
    for root, dirs, files in os.walk(directory):
        for file in files:
            # 检查文件是否以.html 结尾
            if file.endswith('.html'):
                file_path = os.path.join(root, file)
                try:
                    # 删除文件
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

directory_to_clean = "docs"
delete_html_files(directory_to_clean)

Deleted: docs\index.html
Deleted: docs\主观CTA\20170627_20240830_哲萌1号_nav_analysis.html
Deleted: docs\主观CTA\20180509_20240726_旌安1号_nav_analysis.html
Deleted: docs\主观CTA\20180907_20241206_草本优益1号_nav_analysis.html
Deleted: docs\主观CTA\20190307_20241206_济海实恒对冲二号_nav_analysis.html
Deleted: docs\主观CTA\20201106_20241206_草本致远1号_nav_analysis.html
Deleted: docs\主观价值\20170322_20241018_红筹长线3号_nav_analysis.html
Deleted: docs\主观价值\20210625_20241025_睿郡财富11号1期_nav_analysis.html
Deleted: docs\套利\20170512_20241206_远澜翠柏1号_nav_analysis.html
Deleted: docs\套利\20230209_20241129_辰元优选稳健_nav_analysis.html
Deleted: docs\量化CTA\20151104_20241206_远澜云杉_nav_analysis.html
Deleted: docs\量化CTA\20151210_20240524_千象1期_nav_analysis.html
Deleted: docs\量化CTA\20170217_20241206_宏锡量化CTA7号_nav_analysis.html
Deleted: docs\量化CTA\20170301_20241206_均成CTA1号_nav_analysis.html
Deleted: docs\量化CTA\20190228_20241206_会世泰和CTA1号_nav_analysis.html
Deleted: docs\量化CTA\20200910_20241206_弘源多元化CTA_nav_analysis.html
Deleted: docs\量化CTA\20240103_202

In [7]:
# 生成新的html文件
for row in basic_info.itertuples(index=False, name=None):
    nav_data_path = os.path.join("data",f"{row[2]}净值20241224.xlsx")
    demo = NavResearch(nav_data_path,row[0],row[1],row[2],row[3],row[4],row[5])
    demo.get_data()
    demo.get_analysis_table()
    demo.get_html()

草本优益1号无法推断频率,自动转为周度
济海实恒对冲二号无法推断频率,自动转为周度
均成CTA1号无法推断频率,自动转为周度
弘源多元化CTA无法推断频率,自动转为周度
宏锡量化CTA7号无法推断频率,自动转为周度
会世元丰CTA1号无法推断频率,自动转为周度
远澜翠柏1号无法推断频率,自动转为周度


In [8]:
# 生成新的index.html文件
def generate_index_html(folder_path: Path):
    # 获取目录及其子目录中的所有 HTML 文件
    html_files = folder_path.rglob("*.html")
    print(html_files)

    # 创建一个新的 index.html 文件
    with open(folder_path.joinpath("index.html"), "w", encoding="utf-8") as f:
        f.write(
            """<html>
<head>
<meta charset="UTF-8">
<title>Value over Time</title>
<style>
    body {
        font-family: Arial, sans-serif;
        font-size: 20px;
        color: blue;
    }
    h2 {
        font-size: 25px;
        color: blue;

    }
    a {
        text-decoration: none;
    }
</style>
</head>
<body>"""
        )
        # 创建一个集合来存储已经处理过的文件夹，避免重复创建链接
        processed_folders = set()
        # 为每个 HTML 文件创建一个链接
        for html_file in html_files:
            if html_file.name == "index.html":
                continue
            relative_path = html_file.relative_to(folder_path)
            # 如果文件夹还没有被处理过，为文件夹创建一个链接
            if html_file.parent not in processed_folders:
                f.write(f"<h2><a>{html_file.parent.name}</a></h2>\n")
                processed_folders.add(html_file.parent)

            # 创建HTML文件的链接
            f.write(
                f'<a href="{relative_path}">{relative_path.name.replace("_nav_analysis","")}</a><br/>\n'
            )

        f.write("</body>\n</html>")

# 使用函数
generate_index_html(Path("docs"))

<generator object Path.rglob at 0x000001A38408B780>
